# Install, Paths and Parameters

In [6]:
import os
from pathlib import Path
import getpass
import numpy as np
import time
import torch
from torch import nn
from tqdm import tqdm
from torch.utils.data import DataLoader
import random
import sys

# allow imports when running script from within project dir
[sys.path.append(i) for i in ['.', '..']]

# local
from src.helpers.helpers import get_random_indexes, get_random_classes
from src.model.dino_model import get_dino, ViTWrapper
from src.model.data import *
from src.model.eval import validate_network_dino

# Custom imports
import torchattacks
from torchattacks import *
import torch.optim as optim
from torchvision import transforms as pth_transforms
from torchvision.utils import save_image

# seed
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

username = getpass.getuser()
DATA_PATH = Path('/','cluster', 'scratch', 'thobauma', 'dl_data')
MAX_PATH = Path('/','cluster', 'scratch', 'mmathys', 'dl_data')

# Image Net
ORI_PATH = Path(DATA_PATH, 'ori_data')
CLASS_SUBSET_PATH = Path(ORI_PATH, 'class_subset.npy')

VAL_PATH = Path(ORI_PATH, 'validation')
VAL_IMAGES_PATH = Path(VAL_PATH,'images')
VAL_LABEL_PATH = Path(VAL_PATH, 'correct_labels.txt')

In [7]:
INDEX_SUBSET = None
NUM_WORKERS= 0
PIN_MEMORY=True

BATCH_SIZE = 80

DEVICE = 'cuda'

In [8]:
#!python ../setup/collect_env.py

In [9]:
model, linear_classifier = get_dino(model_name='vit_small', patch_size=16, n_last_blocks=4, avgpool_patchtokens=False, device=DEVICE)

Please use the `--pretrained_weights` argument to indicate the path of the checkpoint to evaluate.
Since no pretrained weights have been provided, we load the reference pretrained DINO weights.
Model vit_small built.
Embed dim 1536
We load the reference pretrained linear weights.


# Load data

In [10]:
from torchvision import transforms as pth_transforms

val_dataset = ImageDataset(VAL_IMAGES_PATH, VAL_LABEL_PATH, NO_NORM_TRANSFORM, None, index_subset=None)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, shuffle=False)

NameError: name 'NO_NORM_TRANSFORM' is not defined

# Wrap model

In [ ]:
model_wrap = ViTWrapper(model, linear_classifier, DEVICE, n_last_blocks=4, avgpool_patchtokens=False)
model_wrap= model_wrap.to(DEVICE)

# Attack model and generate datasets

In [ ]:
attacks = [
    #(PGD(model_wrap, eps=0.06, alpha=0.015, steps=20), 'pgd_06'),
    #(CW(model_wrap, c=50, lr=0.0031, steps=30), 'cw'),
    #(PGDL2(model_wrap, eps=0.03, alpha=0.015, steps=20), 'pgd_l2_03')
    (FGSM(model_wrap, eps=0.06), 'fgsm_06')
]

In [ ]:
total = len(val_loader.dataset)

for atk, name in attacks:
    
    print("-"*70)
    print(atk)
    
    STORE_PATH = Path(MAX_PATH, f'adversarial_data/{name}')
    STORE_LABEL_PATH = Path(STORE_PATH, 'validation/labels.txt')
    STORE_IMAGES_PATH = Path(STORE_PATH, 'validation/images/')
    adv_labels = {}
    
    if not os.path.exists(STORE_IMAGES_PATH):
        os.makedirs(STORE_IMAGES_PATH)

    correct = 0
    clean_correct = 0
    start = time.time()
    
    for images, labels, img_names in tqdm(val_loader):
        
        labels = labels.to(DEVICE)
        
        adv_images = atk(images, labels)
        
        with torch.no_grad():
            outputs = model_wrap(adv_images)

        _, pre = torch.max(outputs.data, 1)

        correct += (pre == labels).sum()
        
        for i in range(adv_images.shape[0]):
            save_image(adv_images[i,:,:,:], Path(STORE_IMAGES_PATH, Path(img_names[i])))
            adv_labels[img_names[i]] = labels.cpu().numpy()[i]
            
        del images
        del adv_images
        del labels
        torch.cuda.empty_cache()

    print('Total elapsed time (sec): %.2f' % (time.time() - start))
    print('Accuracy against attack: %.2f %%' % (100 * float(correct) / total))
    
    df = pd.DataFrame.from_dict(adv_labels, orient='index')
    df.to_csv(STORE_LABEL_PATH, sep=" ", header=False)